In [7]:
import pandas as pd
from ydata_profiling import ProfileReport
treino = pd.read_csv('train.csv')
teste = pd.read_csv('test.csv')


In [ ]:
# Gera o report e grava em html
profile = ProfileReport(treino, title="titanic_treino")
profile.to_file("titanic_treino.html")

In [14]:
# verificar infos 
display(treino.info())
display(teste.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


None

In [15]:
#tipos de dados
treino.dtypes.value_counts()

int64      5
object     5
float64    2
Name: count, dtype: int64

In [18]:
display(treino.isnull().sum().sort_values(ascending=False).head(5))
display(teste.isnull().sum().sort_values(ascending=False).head(5))

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
dtype: int64

Cabin          327
Age             86
Fare             1
PassengerId      0
Pclass           0
dtype: int64

In [19]:
#mostra qntd de valores não unicos
treino.nunique().sort_values(ascending=False).head(5)

PassengerId    891
Name           891
Ticket         681
Fare           248
Cabin          147
dtype: int64

In [26]:
teste = teste.drop(['Name','Ticket','Cabin'],axis=1)
treino = treino.drop(['Name','Ticket','Cabin'],axis=1)
# axis = 1 collumns axis = 0 rows

In [25]:
treino.loc[treino.Age.isnull(), "Age"] = treino.Age.mean()
teste.loc[teste.Age.isnull(), "Age"] = teste.Age.mean()
teste.loc[teste.Fare.isnull(),'Fare'] = teste.Fare.mean()
treino.loc[treino.Embarked.isnull(),'Embarked'] = treino.Embarked.mode()[0]
#mode mostra a moda

In [27]:
display(treino.isnull().sum().sort_values(ascending=False).head(5))
display(teste.isnull().sum().sort_values(ascending=False).head(5))

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
dtype: int64

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
dtype: int64

In [28]:
col_treino_nr = treino.columns[treino.dtypes != 'object']
treino_nr = treino.loc[:,col_treino_nr]
col_teste_nr = teste.columns[teste.dtypes != 'object']
teste_nr = teste.loc[:,col_teste_nr]

In [29]:
from sklearn.model_selection import train_test_split
X = treino_nr.drop(['PassengerId','Survived'],axis=1)
y = treino.Survived

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

In [30]:
from sklearn import tree
clf_ac = tree.DecisionTreeClassifier(random_state=42)
clf_ac = clf_ac.fit(X_train,y_train)
y_pred_ac = clf_ac.predict(X_val)

In [31]:
from sklearn.neighbors import KNeighborsClassifier
clf_knn = KNeighborsClassifier(n_neighbors=3)
clf_knn = clf_knn.fit(X_train,y_train)
y_pred_knn = clf_knn.predict(X_val)

In [32]:
from sklearn.linear_model import LogisticRegression
clf_rl = LogisticRegression(random_state=42)
clf_rl = clf_rl.fit(X_train,y_train)
y_pred_rl = clf_rl.predict(X_val)

In [33]:
from sklearn.metrics import accuracy_score
display("Arvore", accuracy_score(y_val, y_pred_ac))
display("KNN", accuracy_score(y_val, y_pred_knn))
display("RL", accuracy_score(y_val, y_pred_rl))

'Arvore'

0.6169491525423729

'KNN'

0.6610169491525424

'RL'

0.7254237288135593

In [35]:
from sklearn.metrics import confusion_matrix
display("arvore",confusion_matrix(y_val, y_pred_ac))
display("KNN",confusion_matrix(y_val, y_pred_knn))
display("RL",confusion_matrix(y_val, y_pred_rl))

# matriz de confusão
# Verdadeiro positivo // Falso Negativo 
# Falso positivo // Verdadeiro Negativo

# Acuracy = sum diag principal / total

'arvore'

array([[125,  50],
       [ 63,  57]])

'KNN'

array([[133,  42],
       [ 58,  62]])

'RL'

array([[156,  19],
       [ 62,  58]])

In [ ]:
X_teste = teste_nr.drop("PassengerId",axis=1)
y_pred = clf_rl.predict(X_teste)
teste['Survived'] = y_pred
base_envio = teste[['PassengerId','Survived']]
base_envio.to_csv('resultados2.csv',index=False)